In [ ]:
# CoNLL-U (un format standard pour annoter des corpus linguistiques)
def read_conllu_file(filepath):
    # contiendra toutes les phrases
    sentences = []
    # temporaire, utilisée pour construire une phrase à la fois
    sentence = []
    #Lit le fichier ligne par ligne et enlève les espaces et retours à la ligne.
    with open(filepath, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            # on ignore les commentaires
            if line.startswith("#") or line == "":
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            # Les lignes valides sont découpées en colonnes
            else:
                parts = line.split('\t')
                # recupération du mot et sont etiquete
                if len(parts) == 10:
                    word = parts[1]
                    pos_tag = parts[3]
                    sentence.append((word, pos_tag))
        if sentence:
            sentences.append(sentence)
    return sentences

from collections import defaultdict

def create_dictionaries_fr(sentences):
    #  (tag, mot) : Combien de fois un mot a été vu avec un certain tag
    emission_counts = defaultdict(int)
    # (prev_tag, tag) :Combien de fois un tag suit un autre tag
    transition_counts = defaultdict(int)
    # (tag) : Combien de fois chaque tag a été vu au total
    tag_counts = defaultdict(int)
    # calculer les proba d'emission et de trasition
    for sent in sentences:
        prev_tag = '--s--'
        tag_counts[prev_tag] += 1  # Ajouter --s-- dans le comptage des tags
        for word, tag in sent:
            transition_counts[(prev_tag, tag)] += 1
            emission_counts[(tag, word.lower())] += 1
            tag_counts[tag] += 1
            prev_tag = tag

    return emission_counts, transition_counts, tag_counts
# sert a garder que les mots frequents
def build_vocab(emission_counts, min_freq=2):
    word_freq = defaultdict(int)
    # calculer la frequence des mots
    for (tag, word), count in emission_counts.items():
        word_freq[word] += count

    # On ne garde que les mots qui apparaissent au moins min_freq fois
    frequent_words = [word for word, freq in word_freq.items() if freq >= min_freq]

    # On trie les mots retenus (optionnel) puis on leur assigne un indice unique
    vocab = {word: i for i, word in enumerate(sorted(frequent_words))}
    return vocab


import numpy as np
# calculer les probabilités de transition entre les tags
# paramètre de lissage de Laplace (évite des zéros dans la matrice)
def create_transition_matrix_fr(alpha, tag_counts, transition_counts):
    tags = sorted(tag_counts.keys())
    num_tags = len(tags)
    tag_to_index = {tag: i for i, tag in enumerate(tags)}

    A = np.zeros((num_tags, num_tags))

    for prev_tag in tags:
        for curr_tag in tags:
            i, j = tag_to_index[prev_tag], tag_to_index[curr_tag]
            count = transition_counts.get((prev_tag, curr_tag), 0)
            count_prev = tag_counts[prev_tag]
            A[i, j] = (count + alpha) / (count_prev + alpha * num_tags)

    return A, tag_to_index

# calcule la proba qu'un mot soit emis par un tag
def create_emission_matrix_fr(alpha, tag_counts, emission_counts, vocab):
    tags = sorted(tag_counts.keys())
    # nb de lignes de la matrice
    num_tags = len(tags)
    # nb de colonnes de la matrice
    num_words = len(vocab)

    tag_to_index = {tag: i for i, tag in enumerate(tags)}
    word_to_index = vocab

    B = np.zeros((num_tags, num_words))
    # parcourir les paires et ceux qui ne sont pas dans le vocab on les ignore
    for (tag, word), count in emission_counts.items():
        if word not in word_to_index:
            continue
        #  calcule de proba avec le lissage
        i = tag_to_index[tag]
        j = word_to_index[word]
        B[i, j] = (count + alpha) / (tag_counts[tag] + alpha * num_words)

    # gestion des valeurs nuls
    # pour les cases nuls on applique que le lissage pour calculer la prob
    for tag in tags:
        i = tag_to_index[tag]
        for word in vocab:
            j = word_to_index[word]
            if B[i, j] == 0:
                B[i, j] = alpha / (tag_counts[tag] + alpha * num_words)

    return B, tag_to_index, word_to_index

import math
# ========== Viterbi Forward ==========
def viterbi_forward_fr(A, B, test_sentence, tag_to_index, word_to_index):
    num_tags = len(tag_to_index)       # Nombre total de tags
    m = len(test_sentence)            # Longueur de la phrase à étiqueter

    # Initialisation de la matrice des meilleures probabilités et des meilleurs chemins
    best_probs = np.full((num_tags, m), -np.inf)   # Probabilité log de chaque tag à chaque position
    best_paths = np.zeros((num_tags, m), dtype=int)  # Indice du meilleur tag précédent

    # Récupération de l’indice du tag de début (--s--)
    s_idx = tag_to_index['--s--']

    # Traitement du premier mot
    first_word = test_sentence[0].lower()
    word_index = word_to_index.get(first_word, None)

    # Initialisation de la première colonne avec les transitions depuis le tag de début
    for i in range(num_tags):
        if A[s_idx, i] > 0:  # S'assurer que la transition est possible
            emit_prob = B[i, word_index] if word_index is not None else 1e-6  # Gestion des mots inconnus
            best_probs[i, 0] = math.log(A[s_idx, i]) + math.log(emit_prob)  # Log probabilité initiale

    # Remplissage de la matrice best_probs et best_paths pour chaque mot suivant
    for t in range(1, m):
        word = test_sentence[t].lower()
        word_index = word_to_index.get(word, None)

        for j in range(num_tags):  # Tag courant
            max_prob = -np.inf
            best_k = 0
            for k in range(num_tags):  # Tag précédent
                trans_prob = A[k, j]  # Probabilité de transition de k vers j
                emit_prob = B[j, word_index] if word_index is not None else 1e-6  # Émission avec fallback
                prob = best_probs[k, t-1] + math.log(trans_prob) + math.log(emit_prob)  # Probabilité cumulée

                # Mise à jour si meilleure probabilité trouvée
                if prob > max_prob:
                    max_prob = prob
                    best_k = k
            best_probs[j, t] = max_prob  # Meilleure probabilité pour tag j à position t
            best_paths[j, t] = best_k    # Meilleur tag précédent menant à ce tag

    return best_probs, best_paths

# ========== Viterbi Backward ==========
def viterbi_backward_fr(best_probs, best_paths, tag_to_index, states):
    m = best_probs.shape[1]  # Longueur de la phrase

    z = [None] * m       # Indices des tags prévus
    pred = [None] * m    # Liste finale des prédictions de tags (sous forme de string)

    # On commence par le tag ayant la plus grande probabilité à la dernière position
    last_tag = np.argmax(best_probs[:, m-1])
    z[m-1] = last_tag
    pred[m-1] = states[last_tag]  # Conversion de l’indice en tag

    # On remonte la chaîne en suivant les meilleurs chemins sauvegardés
    for i in reversed(range(1, m)):
        z[i-1] = best_paths[z[i], i]     # Récupération de l’indice du meilleur tag précédent
        pred[i-1] = states[z[i-1]]       # Conversion en tag string

    return pred  # Liste des tags prédits


In [ ]:
import random

def evaluate_model_accuracy(test_sentences, A, B, tag_to_index, word_to_index):
    # Liste des états possibles (étiquettes), triés pour assurer un ordre cohérent
    states = sorted(tag_to_index.keys())

    # Initialisation des compteurs de bonnes prédictions et de total de mots
    correct = 0
    total = 0

    # Parcours de chaque phrase du jeu de test
    for sentence in test_sentences:
        # Séparation des mots et des étiquettes réelles
        words = [word for word, true_tag in sentence]
        true_tags = [true_tag for word, true_tag in sentence]

        # Calcul des probabilités et des chemins via l'algorithme Viterbi (étape forward)
        best_probs, best_paths = viterbi_forward_fr(A, B, words, tag_to_index, word_to_index)

        # Récupération des étiquettes prédites via backtracking (étape backward)
        pred_tags = viterbi_backward_fr(best_probs, best_paths, tag_to_index, states)

        # Comparaison des étiquettes prédites avec les vraies pour chaque mot
        for pred_tag, true_tag in zip(pred_tags, true_tags):
            if pred_tag == true_tag:
                correct += 1  # Bonne prédiction
            total += 1      # Incrément du total

    # Calcul du taux de précision (accuracy) sur tout le corpus
    accuracy = correct / total if total > 0 else 0
    return accuracy


def test_random_sentence(test_sentences, A, B, tag_to_index, word_to_index):
    states = sorted(tag_to_index.keys())

    # Choisir une phrase aléatoire du jeu de test
    sentence = random.choice(test_sentences)
    words = [word for word, tag in sentence]
    true_tags = [tag for word, tag in sentence]

    print("=== Phrase testée ===")
    print(" ".join(words))

    # Appliquer Viterbi sur cette phrase pour obtenir les étiquettes prédites
    best_probs, best_paths = viterbi_forward_fr(A, B, words, tag_to_index, word_to_index)
    pred_tags = viterbi_backward_fr(best_probs, best_paths, tag_to_index, states)

    # Affichage des résultats
    print("\nMots :")
    print(words)
    print("\nÉtiquettes réelles :")
    print(true_tags)
    print("\nÉtiquettes prédites :")
    print(pred_tags)

    # Calcul et affichage de l'accuracy pour cette phrase uniquement
    correct = sum(p == t for p, t in zip(pred_tags, true_tags))
    total = len(true_tags)
    accuracy = correct / total if total > 0 else 0
    print(f"\nAccuracy sur cette phrase : {accuracy:.2%}")


In [ ]:

# 1. Charger les données d'entraînement
train_sentences = read_conllu_file("/content/train.conllu")

# 2. Créer les dictionnaires à partir de l'entraînement
emission_counts, transition_counts, tag_counts = create_dictionaries_fr(train_sentences)

# 3. Construire le vocabulaire
vocab = build_vocab(emission_counts)

# 4. Créer les matrices A (transition) et B (émission)
alpha = 0.001  # paramètre de lissage
A, tag_to_index = create_transition_matrix_fr(alpha, tag_counts, transition_counts)
B, tag_to_index, word_to_index = create_emission_matrix_fr(alpha, tag_counts, emission_counts, vocab)

# 5. Charger les données de test
test_sentences = read_conllu_file("/content/test.conllu")

# 6. Évaluer le modèle
acc = evaluate_model_accuracy(test_sentences, A, B, tag_to_index, word_to_index)
print(f"Accuracy du modèle sur tout le test set : {acc:.2%}")

# 7. Tester une phrase aléatoire
test_random_sentence(test_sentences, A, B, tag_to_index, word_to_index)


Accuracy du modèle sur tout le test set : 91.23%
=== Phrase testée ===
Comme en 2007 , le club local d' Arsenal remporte le trophée .

Mots :
['Comme', 'en', '2007', ',', 'le', 'club', 'local', "d'", 'Arsenal', 'remporte', 'le', 'trophée', '.']

Étiquettes réelles :
['COSUB', 'PREP', 'CHIF', 'PUNCT', 'DETMS', 'NMS', 'ADJMS', 'PREP', 'PROPN', 'VERB', 'DETMS', 'NMS', 'YPFOR']

Étiquettes prédites :
['COSUB', 'PREP', 'CHIF', 'PUNCT', 'DETMS', 'NMS', 'ADJMS', 'PREP', 'PROPN', 'VERB', 'DETMS', 'NMS', 'YPFOR']

Accuracy sur cette phrase : 100.00%
